# Create a notebook to perform Cash-Karp Runge-Kutta integration for multiple coupled variables with adaptive stepwise control

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Define our coupled derivatives

In [ ]:
def dydx(x,y):
    # y is multivariable, y[0]=y, y[1]=z
    y_derivs = np.zeros(2)    # 2 because 2 variables. y_derivs[0]=dy/dx, y_derivs[1]=dz/dx 
    y_derivs[0] = y[1]        # dy/dx = z (given)
    y_derivs[1] = -1*y[0]     # dz/dx = -y (given)
    return y_derivs

# Define the Cash-Karp Runge-Kutta method

In [ ]:
def ck_rk_mv_core(dydx,xi,yi,nv,h):  # gives updated estimate of y,z after a step h using cash-karp runge-kutta
    # dy/dx is the above function 
    # xi,yi is x,y at left side of step
    # nv is the number of variables (nv=2 for this problem)
    # h is step size
    
    # k arrays defined in 4th order RK method (k? means k1,k2,...,k9)
    k1 = np.zeros(nv)       # in this case, since nv=2, k1[0] corresponds to y, k1[1] corresponds to z
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    y_temp = np.zeros(nv)   # temprorary y array 
    
    #get k1 values (see slide 4 of lecture 15)
    y_derivs = dydx(xi,yi)  # use the function that we made earlier
    k1[:] = h*y_derivs[:]   # colon is to make sure we use both y and z
    
    # get k2 values
    c2 = 1/5
    a21 = 1/5         # slide 5 of Lecture 15
    y_temp[:] = yi[:] + a21*k1[:] # update y_temp after first step, note we are doing both y and z simultaneously
    y_derivs = dydx(xi+c2*h,y_temp)
    k2[:] = h*y_derivs[:]
    
    # get k3 values
    c3 = 3/10
    a31 = 3/40
    a32 = 9/40
    y_temp[:] = yi[:] + a31*k1[:] + a32*k2[:]
    y_derivs = dydx(xi + c3*h,y_temp)
    k3[:] = h*y_derivs[:]

    # get k4 values
    c4 = 3/5
    a41 = 3/10
    a42 = -9/10
    a43 = 6/5
    y_temp[:] = yi[:] + a41*k1[:] + a42*k2[:] + a43*k3[:]
    y_derivs = dydx(xi + c4*h,y_temp)
    k4[:] = h*y_derivs[:]
    
    # get k5 values
    c5 = 1
    a51 = -11/54
    a52 = 5/2
    a53 = -70/27
    a54 = 35/27
    y_temp[:] = yi[:] + a51*k1[:] + a52*k2[:] + a53*k3[:] + a54*k4[:]
    y_derivs = dydx(xi + c5*h,y_temp)
    k5[:] = h*y_derivs[:]
    
    # get k6 values
    c6 = 7/8
    a61 = 1631/55296
    a62 = 175/512
    a63 = 575/13824
    a64 = 44275/110592
    a65 = 253/4096
    y_temp[:] = yi[:] + a61*k1[:] + a62*k2[:] + a63*k3[:] + a64*k4[:] + a65*k5[:]
    y_derivs = dydx(xi + c6*h,y_temp)
    k6[:] = h*y_derivs[:]    # note to self: I could put all of the k's into one matrix rather than having a seperate variable for each k. maybe do later if bored
    
    # advance y by a step h
    b = np.array([37/378 ,0 ,250/621 ,125/594 ,0 ,512/1771])
    b_star = np.array([2825/27648 ,0 ,18575/48384 ,13525/55296 ,277/14336 ,1/4])
    yipo = yi + b[0]*k1 + b[1]*k2 + b[2]*k3 + b[3]*k4 + b[4]*k5 + b[5]*k6 
    yipo_star = yi + b_star[0]*k1 + b_star[1]*k2 + b_star[2]*k3 + b_star[3]*k4 + b_star[4]*k5 + b_star[5]*k6 
    
    delta = np.fabs(yipo - yipo_star)
    return yipo, delta

# Define adaptive step size driver for the Cash-Karp Runge-Kutta method (mv_ad = multi-variable adaptive)

In [ ]:
def ck_rk_mv_ad(dydx,x_i,y_i,nv,h,tol):
    # define safety scale
    SAFETY = 0.9    # make sure the size decrease is at minumum 90%, we don't want step to decrease by more than 90
    H_NEW_FAC = 2.0 # make sure to take at max double the step, limits how far we can increase the step size
    
    # set max number of iterations
    imax=10000
    
    # set iteration variable
    i = 0
    
    # remember the original step
    h_step = h
    
    # find error
    y_2, Delta = ck_rk_mv_core(dydx,x_i,y_i,nv,h_step)

    
    # adjust step
    while(Delta.max()/tol > 1.0):    # while the max Delta is greater than the tolerance. note: uses newton-rapson root finding to divide numebrs
        h_step *= SAFETY * (Delta.max()/tol)**(-0.25)   # SAFETY is to make sure we decrease h_step, the other term 
                                                            # will be less than 1, but will most likey be close to 1, 
                                                            # we don't want it to be .99999 so the SAFETY guraantees we at 
                                                            # least decrease by 10%
        y_2, Delta = ck_rk_mv_core(dydx,x_i,y_i,nv,h_step)
        # make sure we don't get infinite loop by checking iteration
        if(i>=imax):
            print('Too many iterations in ck_rk_mv_ad()')
            raise StopIteration('Ending after i = ',i)
        i += 1
        
    # For the next step, try to take a bigger step regardless of if the previous while loop did anything
    # note: once we get to this point (Delta.max()/tol) <= 1 since we have to exit the while loop
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9),h_step*H_NEW_FAC)
    
    # y_2 is from the newest h_step, h_new is the h we should use for the next iteration, h_step is the newest h_step 
    return y_2, h_new, h_step
            
        
        

# Define a wrapper for the Cash-Karp Runge-Kutta method

In [ ]:
def ck_rk_mv(dfdx,a,b,y_a,tol):
    # dfdx is the derivative wrt x (copied that from slide, idk what wrt x means though)
    # a,b is the lower bound,upper bound
    # y_a are the initial conditions. (the boundary conditions)
    # tol is the tolerance for integrating y
    
    xi = a          # starting step
    yi = y_a.copy()
    
    h = 1.0e-4 * (b-a)   # starting h, our adaptive function will further change this. Start small
                            # note: this might work fine if integrating backwards, it will just be a negative h
    
    imax = 10000   # reasonably we will probably never reach this, it is here however to prevent infinite loops
    
    i = 0    # set our iteration variable
    
    nv = len(y_a)  # number of dimensions used will be the length of y_a
    
    x = np.full(1,a)        # why not np.array([a]) is it to just match syntax of y below?
    y = np.full((1,nv),y_a) # 1 x nv matrix to put our different variables into
    
    flag = 1   # arbitrary switch for the while loop. Obviously switched on to run the the loop. Note: 1 = on, 0 = off
    
    while(flag):
        yi_new, h_new, h_step = ck_rk_mv_ad(dydx, xi, yi, nv, h, tol)  # see function above to know what this does
        
        h = h_new  # replace our original guess above
        
        # once we reach the end, we want to prevent an overshoot
        if(xi+h_step>b):
            h = b-xi   # make the last step perfectly reach b
            
            yi_new, h_new, h_step = ck_rk_mv_ad(dydx, xi, yi, nv, h, tol) # recalc y_i+1
            
            flag = 0  # since we will only be here when we reach the end, turn off the while loop
            
        # if we have not reached the end, this is how we do the next step
        
        xi += h_step        # change xi,yi with the output of rk4_mv_ad
        yi[:] = yi_new[:]
        
        x = np.append(x,xi) # add the new x to the x_array
        
        # it's a little more complicated to set up adding the new i. This is because we are changing the dimensions
        y_new = np.zeros((len(x),nv))  # set up the shell of y
        y_new [0:len(x)-1,:] = y       # fill the first part of the shell with the previous y values to preserve them
        y_new[-1,:] = yi[:]            # put our newest value at the end of the shell
        del y                          # delete y in order to
        y = y_new                      # replace y with the now filled shell
        
        if(i>=imax):    # prevent inifinite loop
            print('Maximum iterations reached.')
            raise StopIteration('Max iterations set to ',i)
            
        i += 1   # end of the first step, so increment i and go through the loop again until the flag above
        
        s = 'i = %3d\tx = %9.8f\th = %9.8f\tb = %9.8f' % (i,xi,h_step,b)
        print(s)   # print each step.
        if(xi==b):
            flag = 0 # makes sure to turn off the while loop
    return x,y  # return answer

In [ ]:
a = 0.0           # integration limits (range where we want to find diff eq)
b = 2.0 * np.pi

y_0 = np.array([0.0,1.0]) # initial conditions y(0)=0 and z(0)=1, recall y array is [y,z]

tolerance = 1.0e-6

# time to use our function
x,y = ck_rk_mv(dydx,a,b,y_0,tolerance)

In [ ]:
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label=r'z(x) = $\frac{dy}{dx}(x)$')
x_graph = np.linspace(a,b,1000)
plt.plot(x_graph,np.sin(x_graph),label='sin(x)')
plt.plot(x_graph,np.cos(x_graph),label='cos(x)')
plt.legend()
plt.show()

In [ ]:
y_error = y[:,0] - np.sin(x)
dydx_error = y[:,1] - np.cos(x)

plt.plot(x,y_error, label='y(x) Error')
plt.plot(x,dydx_error, label=r'$\frac{dy}{dx}$ Error')
plt.legend()
plt.show()